# Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

Objetivo del Ejercicio: Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos que deberas seguir son:

1. Preparación de Datos: 

    - Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

2. Análisis Descriptivo: 

    - Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

3. Prueba Estadística:

    - Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

### Importamos las librerías que necesitamos

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from src import support as sp

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
# Cargamos los datos generados en la Fase 1
df_combined = pd.read_csv('../data/df_combined.csv', index_col=0)

In [6]:
# Mostramos las primeras filas del DataFrame
df_combined.head(3) 

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN


- Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

* Si solo filtramos por esas 2 columnas, luego perdemos la referencia de quien son estos clientes, por lo cuál, filtraremos por 3 columnas

In [8]:
# Filtrar el conjunto de datos para incluir las columnas 'loyalty_number', 'flights_booked' y 'education'
df_filtered = df_combined[['loyalty_number', 'flights_booked', 'education']]
df_filtered.shape

(401688, 3)

- Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [24]:
# Agrupamos por loyalty_number y education, que son los valores que se repiten, para extraer las estadísticas descriptivas de flights_booked por cliente.
df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].describe().sample(10)

,,count,mean,std,min,25%,50%,75%,max
loyalty_number,education,,,,,,,,
548228,Bachelor,24.0,6.833333,5.880452,0.0,1.50,6.5,11.25,20.0
273353,Master,24.0,2.166667,4.593631,0.0,0.00,0.0,0.00,16.0
884433,Bachelor,24.0,5.375000,5.475737,0.0,0.00,4.5,9.00,21.0
424983,College,24.0,7.750000,5.620227,0.0,3.75,9.0,11.00,17.0
465570,Bachelor,24.0,5.666667,5.623450,0.0,0.00,5.0,10.25,20.0
205697,Master,24.0,5.083333,6.261800,0.0,0.00,1.5,9.00,19.0
357166,Bachelor,24.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
787875,College,24.0,6.250000,5.780402,0.0,1.00,6.0,9.25,20.0
210100,Bachelor,24.0,0.791667,2.042572,0.0,0.00,0.0,0.00,7.0


- Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

In [27]:
# Primero filtramos el conjunto de datos 
df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].sum().reset_index().sample(10)

,loyalty_number,education,flights_booked
9667,618022,Bachelor,113
9488,609296,Bachelor,144
12713,780631,Bachelor,11
15869,952730,Doctor,152
8855,575849,Bachelor,97
2169,216359,High School or Below,10
9097,588015,Bachelor,111
8193,541008,Bachelor,52
13663,833322,College,129
11370,709948,Bachelor,157


In [30]:
# Guardamos este DataFrame en una variable para usarlo en nuestro A/B testing
df_filtered_ab = df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].sum().reset_index()
df_filtered_ab.tail(3)

,loyalty_number,education,flights_booked
16734,999940,Bachelor,63
16735,999982,College,22
16736,999986,Bachelor,121


In [31]:
df_filtered_ab.education.value_counts() # Miramos como están distribuidos los niveles de educación

education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64

El orden de los niveles de educación de menor a mayor es:

1. **High School or Below** (Secundaria o menos)
2. **College** (Universidad, generalmente implica que no se completó un título universitario)
3. **Bachelor** (Licenciatura)
4. **Master** (Máster)
5. **Doctor** (Doctorado)

### Ahora vamos a crear nuestro test_group, categorizando la columna education en 2 grupos: 
    * GRUPO A: Clientes con nivel de educación inferior: High School or Below y College
    * GRUPO B: Clientes con nivel de educación superior: Doctor y Master